In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from urllib.request import urlopen
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import requests

import re

In [2]:
movie = pd.read_csv('data_ver4.csv')

In [111]:
len(movie)

20528

# 중간 저장

# 시작

In [226]:
movie_df = pd.read_csv('data_ver4.csv')

# 1. 중복된 영화 찾기

In [215]:
pd.set_option('display.max_rows', 10000)

# 중복된 영화 확인하기
name_counts = movie_df['영화명'].value_counts()

In [216]:
# 영화 카운트만 따로 df로 만들기
n_c_df = pd.DataFrame(name_counts)

In [217]:
print(n_c_df.iloc[0, :].name)
print(int(n_c_df.iloc[0, :].values))

로미오와 줄리엣
6


In [218]:
# 중복된 영화 리스트

dupl = []

for i in range(len(n_c_df)):
    if int(n_c_df.iloc[i, :].values) > 1:
        dupl.append(n_c_df.iloc[i, :].name)

In [219]:
len(dupl)

483

In [124]:
movie_df['매출액']

0          26000
1         300000
2        1830000
3           4000
4           5000
          ...   
20523    2992500
20524      10000
20525      88000
20526    3543400
20527       4000
Name: 매출액, Length: 20528, dtype: int64

# 2. 감독 따기

In [220]:
directors = []

for i in range(0, len(movie_df)):
    tmp = str(movie_df["감독"][i]).split(',')[0]
    
    directors.append(tmp)    

pd.Series(directors).value_counts() 


movie_df['감독'] = directors

In [121]:
# # 기타 장르 삭제
# movie_df = movie_df[movie_df['장르'] != "기타"]

## 숫자데이터 데이터 타입변환하기(object->int)

In [37]:
nums = []

for num in movie_df['누적매출액']:
    nu_num = num.replace(",", "")
    nums.append(int(nu_num))
    

In [38]:
movie_df['누적매출액'] = nums

In [39]:
nums = []

for num in movie_df['매출액']:
    nu_num = num.replace(",", "")
    nums.append(int(nu_num))
    

In [40]:
movie_df['매출액'] = nums

In [41]:
nums = []

for num in movie_df['관객수']:
    nu_num = num.replace(",", "")
    nums.append(int(nu_num))
    

In [42]:
movie_df['관객수'] = nums

In [43]:
nums = []

for num in movie_df['누적관객수']:
    nu_num = num.replace(",", "")
    nums.append(int(nu_num))
    

In [44]:
movie_df['누적관객수'] = nums

In [45]:
nums = []

for num in movie_df['스크린수']:
    nu_num = num.replace(",", "")
    nums.append(int(nu_num))
    

In [46]:
movie_df['스크린수'] = nums

In [47]:
nums = []

for num in movie_df['상영횟수']:
    nu_num = num.replace(",", "")
    nums.append(int(nu_num))
    

In [48]:
movie_df['상영횟수'] = nums

In [125]:
# 누적 매출액 0 갯수 854
movie_df = movie_df[(movie_df['누적매출액'] > 1) & (movie_df['누적관객수'] > 1)]

# 네이버 영화 감독 정보 크롤링

In [2]:
# 링크 접속
def url_search(movie_name):
    url1 = "https://movie.naver.com/movie/search/result.naver?query={}&section=all&ie=utf8".format(movie_name)
    html1 = requests.get(url1)
    soup1 = BeautifulSoup(html1.text, 'html.parser')
    
    return soup1

In [3]:
def one_or_more(soup1):
    tag_search_child = soup1.find("div", class_ = "search_header first-child")
    if tag_search_child != None:
        tag_h3 = tag_search_child.find('h3').text
        if tag_h3 == "영화":
            if bool(tag_search_child) == True:
                num1 = soup1.find('span', class_ = 'num')
                num2 = num1.get_text()
                num3 = num2[3]

                return int(num3)
            else:
                return "없음"
        else:
            return "없음"
    else:
        return "없음"

In [419]:
soup_ = url_search('합의된 관계')
one_or_more(soup_)

'없음'

## 영화 검색하니 하나짜리 영화

In [4]:
# href 태그 찾기

def href_tag(soup1):

    tag_search_list1 = soup1.find("ul", class_ = "search_list_1")
    
    if bool(tag_search_list1) == True:
        tag_a = soup1.select('div#old_content > ul.search_list_1 > li > p.result_thumb > a')
        global href
        for i in tag_a:
            href = i.attrs['href']
    
        return href
    
    else:
        return "없음"
    

## 영화가 여러 개인 경우

In [5]:
def tag_dt_title(soup1):

    tag_search_list1 = soup1.find("ul", class_ = "search_list_1")
    
    if bool(tag_search_list1) == True:
#         tag_li = tag_search_list1.find_all('li') # 여기 수정
        tag_dt = tag_search_list1.find_all('dt')
#         if bool(tag_li) is True:
#             for li in tag_li:
#                 tag_dt = tag_li.find_all('dt') # 여기도 수정
   
        return tag_dt
    
    else:
        return "없음"
        
# h1 = soup.html.body.h1 

In [6]:
def tag_dd_info(soup1):

    tag_search_list1 = soup1.find("ul", class_ = "search_list_1")
    
    if bool(tag_search_list1) == True:
#         tag_li = tag_search_list1.find_all('li') # 여기 수정
        tag_dd = tag_search_list1.find_all('dd', class_ = 'etc')
        return tag_dd
#         if bool(tag_li) is True:
#             for li in tag_li:
#                 tag_dd = tag_li.find_all('dd', class_ = 'etc') # 여기도 수정
                
#             return tag_dd
    else:
        return "없음"

In [7]:
def dt(dt_title):
    titles = []
    
    if dt_title != "없음":
        for dt in dt_title:
            dt = str(dt.get_text())
            if (dt.find('(') > -1):
                dt = dt[:dt.find('(')-1]
                titles.append(dt)

        return titles
    
    else:
        return "없음"

In [8]:
def dd(dd_info):
    list_dd = []
    list_even_split = []
    
    if dd_info != "없음":
        for dd in dd_info:
            dd = str(dd.get_text())
            dd = dd.split('|')
            list_dd.append(dd)

        list_odd = list_dd[0::2]
#     list_even = list_dd[1::2]
    
#     for i in range(len(list_even)):
#         if len(list_even[i]) < 2:
#             direc = list_even[i][0].replace("감독 : ", "")
#             director = direc[0]
#             tmp1 = [director]
#             list_even_split.append(tmp1)
#         else:
#             direc = list_even[i][0].replace("감독 : ", "")
#             act = list_even[i][1].replace("출연 : ", "")
#             director = direc[0]
#             actors = act[0]
#             tmp2 = [director, actors]
#             list_even_split.append(tmp2)  
        return list_odd
    else:
        return "없음"

In [9]:

def arikkari_director(tags, year_data):
    
    # 링크 찾기
    links = []
    for tag in tags:
        link = "https://movie.naver.com{}".format(tag)
        links.append(link)

    # 링크 파싱 준비
    for link in links:
        html = requests.get(link)  
        soup_html = BeautifulSoup(html.text, 'html.parser')
        dd = soup_html.select('dl > dd > p > span')
        
        # 개봉연도가 있는 영화
        if len(list(dd)) >= 4:
            pattern0 = re.compile(r'\w{4}')
            scraped_year = "".join(re.findall(pattern0, dd[3].get_text()))
            #감독 찾기
            if scraped_year == year_data: # 2020은 year_data
                dd2 = soup_html.find_all('dd')
                directo = dd2[0].get_text().strip()
                
                return directo
            else:
                return "없음"
        else:
            return "없음"

In [722]:
tags1 = ["/bi/mi/basic.naver?code=130932"]
year_data = "2018"

arikkari_director(tags1, year_data)

'없음'

## 페이지 찾아들어가서 파싱

In [10]:
# 해당 페이지 링크 가져오기

def find_link(href):
    
    if href != "없음":
        
        url2 = "https://movie.naver.com{}".format(href)
        html2 = requests.get(url2)
        soup2 = BeautifulSoup(html2.text, 'html.parser')
        tag_div_container = soup2.find("div", id = "container").text
        if tag_div_container == None:
            return "없음"
        else:
            return soup2
    
    else:
        return "없음"

In [11]:
soup1 = url_search('더 수어사이드 스쿼드')
num = one_or_more(soup1)

if num == 1:
    href = href_tag(soup1)
    soup2 = find_link(href)

    find_dir = dd_tag(soup2)
    print(bool(find_dir))
    director1 = find_director(find_dir)

NameError: name 'dd_tag' is not defined

In [12]:
# dd 태그 찾기

def dd_tag(soup2):
    
    if soup2 != "없음":
        find_dir = soup2.find_all('dd')
    
        return find_dir
    
    else:
        return "없음"

In [13]:
# director 찾기
def find_director(find_dir):
    
    if find_dir != "없음":
        words = []
    
        for i in find_dir:
            i = i.get_text()
            tmp = []
            tmp.append(i)
        
            for w in tmp:
                w = w.strip('\n')
                words.append(w)
                director = words[0].split(',')[0]
        
            return director
    
    else:
        return "없음"
            

## 감독값이 결측값인 데이터셋

In [14]:
nan_director = pd.read_csv('data_ver5.csv')

In [245]:
type(nan_director['개봉일'][0])

float

## 문제아 삭제

In [330]:
indexNames = nan_director[nan_director['영화명'] == '합의된 관계'].index
# nan_director.drop(indexNames , inplace=True)

In [247]:
nan_director[nan_director['영화명'] == '코마']

,영화명,개봉일,매출액,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,배급사,등급,장르,감독,배우
3554,코마,2020-11-12,18739450,18739450,2563,2563,46,274,러시아,(주)까멜리아이엔티,15세이상관람가,SF,NaN,NaN


In [15]:
nan_movies = list(nan_director['영화명'])

In [43]:
len(nan_director)

3633

In [16]:
def year_(movie_name):
    open_year = nan_director.loc[nan_director['영화명'] == movie_name]['개봉일']
    year = list(open_year)[0]
#     print(year)
    if year != None:
        if type(year) != float:
            year = year.split('-')[0]
            return year

    else:
        return None
    

In [460]:
year_("합의된 관계")

'2019'

In [17]:
def select_year(info):
    years = []
    if info != None:
        for i in range(len(info)):
            if len(info[i]) <= 4:
                pattern0 = re.compile(r'\w{4}')
                year = re.findall(pattern0, info[i][-1])
                if year != []:
                    years.append(year)
        return years
    else:
        return None

In [18]:
def year_compare(searched_years, year_data):
    for i in range(len(searched_years)):
        year = "".join(searched_years[i])
        if year == year_data:
            return year


In [839]:
[['2016'], ['2016'], ['관련영화', '2011']]

list_ = [['1985'], ['2013'], []]
year_data = "1988"
a = year_compare(list_, year_data)
print(a)

None


In [19]:
def arikkari(soup5):
    
    items = soup5.find("ul", {"class": "search_list_1"})
    li_items = items.find_all('li')
    item_dict = []
    for li in li_items:
        
        name_tag = li.find('dt')
        name = name_tag.text
        movie_url = name_tag.find('a')['href']
        item_dict.append(movie_url)
            
    return item_dict

In [674]:
soup = url_search('코마')

name_tag = hrefs(soup)
print(name_tag)

['/movie/bi/mi/basic.naver?code=197277', '/movie/bi/mi/basic.naver?code=186459', '/movie/bi/mi/basic.naver?code=138742', '/movie/bi/mi/basic.naver?code=54338', '/movie/bi/mi/basic.naver?code=39380']


# 실행함수

In [44]:
def directors(movie_name):
    
#     directors = []

#     for movie_name in movie_names:
        soup1 = url_search(movie_name)
        num = one_or_more(soup1)
        
        if num == 1:
            href = href_tag(soup1)
            soup2 = find_link(href)
            
            find_dir = dd_tag(soup2)
            if bool(find_dir) == True:
                director1 = find_director(find_dir)
                return director1
            else:
                return "없음"          
            
        elif num == "없음":
            return "없음"
            
        else:
            dt_title = tag_dt_title(soup1)
            dd_info = tag_dd_info(soup1)
            
            titles = dt(dt_title)
            info = dd(dd_info)
            
            
            year_data = year_(movie_name) # 개봉연도
            searched_years = select_year(info)[:5] # 제작연도 
            
            year = year_compare(searched_years, year_data)
            
            if (year_data == None) or (searched_years == None):
                return "개봉연도 없음"
            
            elif year == None:
                href_tags = arikkari(soup1)
                tags = []
                for i in range(0, len(searched_years)):
                    year_diff = {i : int(year_data) - int("".join(searched_years[i]))}
                    
                    for key, value in year_diff.items():
                        if value <= 2:
                            tag = href_tags[key]
                            tags.append(tag)
                            
                director3 = arikkari_director(tags, year_data)
                
                if director3 != None:
                    return director3

                        

            else:
                soup3 = url_search(movie_name + year)
                href2 = href_tag(soup3)
                soup4 = find_link(href2)
                
                find_dir = dd_tag(soup4)
                director2 = find_director(find_dir)
                
                return director2 

                

                        

    

In [115]:
a = directors('나의 고향')

a

'개봉연도 없음'

In [502]:
print(nan_movies[249:255])
print(directors(nan_movies[249:255]))

['내일이 아니면 언제?', '바르팍을 위한 노래', '부르카 복서', '시간위示間威', '제9회 DMZ국제다큐영화제 단체관람2', '제9회 DMZ국제다큐영화제 한국다큐쇼케이스단편1']
1
2
3
4
5
6
(['수 수 흘렝', '프라딥 포크렐', '알카 라구람', '한 솔', '없음', '없음'], 6)


In [875]:
start = time.time()

print(directors(nan_movies))
end = time.time()

print(end-start)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060


3500
3501
3502
3503
3504
3505
3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
(['개봉연도 없음', '개봉연도 없음', '없음', '없음', '개봉연도 없음', '없음', '없음', '개봉연도 없음', '개봉연도 없음', '개봉연도 없음', '개봉연도 없음', '없음', '개봉연도 없음', '없음', '개봉연도 없음', '이루비', '없음', '없음', '두기봉', '없음', '조셉 치미네라', '브래드 개리스', '에릭 짐머만', '없음', '알렉산드르 아라빈', '마크 포스트만', '크리스토퍼 컴튼', '개봉연도 없음', '존 아즈필리쿠에타', '제이미 웨스턴', '데이빗 로저 메이셀먼 ', '브라이언 애브넷-브래드리', '없음', '코린 에그렉즈키', '

## 1) 결측값이 있는 감독 데이터

In [279]:
print("총 수 : ", len(nan_movies))

총 수 :  3633


In [904]:
directors = directors(nan_movies)

In [278]:
len(directors)

TypeError: object of type 'function' has no len()

## 2) 10개씩 나눠서 ㄱ

In [299]:
# director1 = directors(nan_movies[0:360])
# director2 = directors(nan_movies[360:720])
# director3 = directors(nan_movies[720:1080])
# director4 = directors(nan_movies[1080:1440])
# director5 = directors(nan_movies[1440:1800])
# director6 = directors(nan_movies[1800:2160])
# director7 = directors(nan_movies[2160:2520])
# director8 = directors(nan_movies[2520:2880])
# director9 = directors(nan_movies[2880:3240])
# director10 = directors(nan_movies[3240:3600])
# director11 = directors(nan_movies[3600:3621])


In [26]:
# director1 = []
# for name in nan_movies[0:360]:
#     d = directors(name)
#     director1.append(d)

director2 = []
for name in nan_movies[360:720]:
    d = directors(name)
    director2.append(d)
    
director3 = []
for name in nan_movies[720:1080]:
    d = directors(name)
    director3.append(d)
    
director4 = []
for name in nan_movies[1080:1440]:
    d = directors(name)
    director4.append(d)

director5 = []
for name in nan_movies[1440:1800]:
    d = directors(name)
    director5.append(d)
    
director6 = []
for name in nan_movies[1800:2160]:
    d = directors(name)
    director6.append(d)


director7 = []
for name in nan_movies[2160:2520]:
    d = directors(name)
    director7.append(d)
    
    
director8 = []
for name in nan_movies[2520:2880]:
    d = directors(name)
    director8.append(d)

director9 = []
for name in nan_movies[2880:3240]:
    d = directors(name)
    director9.append(d)


director10 = []
for name in nan_movies[3240:3600]:
    d = directors(name)
    director10.append(d)
    

director11 = []
for name in nan_movies[3600:3621]:
    d = directors(name)
    director11.append(d)


In [25]:
len(director1)

360

In [307]:
# print(nan_movies[0:360])
# print(director1)
# print("====================================================================")
# print(nan_movies[360:720])
# print(director2)
# print("====================================================================")
# print(nan_movies[720:1080])
# print(director3)
# print("====================================================================")
# print(nan_movies[1080:1440])
# print(director4)
# print("====================================================================")
# print(nan_movies[1440:1800])
# print(director5)
# print("====================================================================")
# print(nan_movies[1800:2160])
# print(director6)
# print("====================================================================")
# print(nan_movies[2160:2520])
# print(len(director7))
# print("====================================================================")
# print(nan_movies[2520:2880])
# print(len(director8))
# print("====================================================================")
# print(nan_movies[2880:3240])
print(len(director9))
# print("====================================================================")
# print(nan_movies[3240:3600])
# print(director10)
# print("====================================================================")
# print(nan_movies[3600:3621])
# print(director11)

0


In [280]:
# director1_dic = {name:value for name, value in zip(nan_movies[0:360], director1)}
# director2_dic = {name:value for name, value in zip(nan_movies[360:720], director2)}
# director3_dic = {name:value for name, value in zip(nan_movies[720:1080], director3)}
# director4_dic = {name:value for name, value in zip(nan_movies[1080:1440], director4)}
# director5_dic = {name:value for name, value in zip(nan_movies[1440:1800], director5)}
# director6_dic = {name:value for name, value in zip(nan_movies[1800:2160], director6)}
director7_dic = {name:value for name, value in zip(nan_movies[2160:2520], director7)}
# director8_dic = {name:value for name, value in zip(nan_movies[2520:2880], director8)}
# director9_dic = {name:value for name, value in zip(nan_movies[2880:3240], director9)}
# director10_dic = {name:value for name, value in zip(nan_movies[3240:3600], director10)}
# director11_dic = {name:value for name, value in zip(nan_movies[3600:3621], director11)}


In [281]:
director7_dic

{'신녀': '없', '[음악영화제]-제네시스와 레이디 제이의 발라드': '음'}

In [27]:
print("1번 : ", len(director1))
print("2번 : ", len(director2))
print("3번 : ", len(director3))
print("4번 : ", len(director4))
print("5번 : ", len(director5))
print("6번 : ", len(director6))
print("7번 : ", len(director7))
print("8번 : ", len(director8))
print("9번 : ", len(director9))
print("10번 : ", len(director10))
print("11번 : ", len(director11))

1번 :  360
2번 :  360
3번 :  360
4번 :  360
5번 :  360
6번 :  360
7번 :  360
8번 :  360
9번 :  360
10번 :  360
11번 :  21


In [28]:
real_directors = director1 + director2 + director3 + director4 + director5 + director6 + director7 + director8 + director9 + director10 + director11

In [65]:
real_directors2 = director1 + director2 + director3 + director4 + director5 + director6 + director7 + director8 + director9 + director10 + director11

In [85]:
for i in range(len(real_directors2)):
    if real_directors2[i] == "":
        real_directors2[i].replace(" ", "없음")

In [86]:
real_directors2_df = pd.Series(real_directors2)

In [87]:
real_directors2_df.value_counts().head(10)

없음          1358
개봉연도 없음      506
스즈키 켄이치로      58
다나카 켄타로       22
가토 마사키        20
              17
코지마 유키오       17
메트오페라         15
존 류지          11
나기라 켄조         9
dtype: int64

In [47]:
real_directors_df = pd.DataFrame(real_directors)

In [101]:
nan_director['감독'].value_counts().head(10)

없음          1358
개봉연도 없음      506
스즈키 켄이치로      58
다나카 켄타로       22
가토 마사키        20
              17
코지마 유키오       17
메트오페라         15
존 류지          11
나기라 켄조         9
Name: 감독, dtype: int64

In [48]:
nan_director['감독'] = real_directors_df

In [120]:
len(nan_director[nan_director['감독'] == ""])

17

In [ ]:
nan_director[nan_director['감독'] == ""]

In [109]:
pd.set_option('display.max_rows', 10000)

nan_director['감독'].value_counts().head(1000)

없음                                                                                                                                                                                                                                                                          1358
개봉연도 없음                                                                                                                                                                                                                                                                      506
스즈키 켄이치로                                                                                                                                                                                                                                                                      58
다나카 켄타로                                                                                                                                                                              

# 수기 변경

In [110]:
nan_director.head(10)

,영화명,개봉일,매출액,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,배급사,등급,장르,감독,배우
0,나의 고향,NaN,143,143,1,1,1,1,폴란드,NaN,NaN,애니메이션,개봉연도 없음,NaN
1,디스턴스,NaN,143,143,1,1,1,1,폴란드,NaN,NaN,애니메이션,개봉연도 없음,NaN
2,문명사회,NaN,143,143,1,1,1,1,불가리아,NaN,NaN,애니메이션,없음,NaN
3,바다의 법,NaN,143,143,1,1,1,1,불가리아,NaN,NaN,애니메이션,없음,NaN
4,비,NaN,143,143,1,1,1,1,불가리아,NaN,NaN,애니메이션,개봉연도 없음,NaN
5,스노보드,NaN,143,143,1,1,1,1,불가리아,NaN,NaN,애니메이션,없음,NaN
6,어디에나 있는 앨리스,NaN,143,143,1,1,1,1,폴란드,NaN,NaN,애니메이션,없음,NaN
7,언더라이프,NaN,143,143,1,1,1,1,폴란드,NaN,NaN,애니메이션,개봉연도 없음,NaN
8,유행,NaN,143,143,1,1,1,1,불가리아,NaN,NaN,애니메이션,개봉연도 없음,NaN
9,이야기,NaN,143,143,1,1,1,1,불가리아,NaN,NaN,애니메이션,개봉연도 없음,NaN


In [114]:
nan_director.to_csv("data_ver5_1.csv", header = True, index = False, encoding = 'utf-8-sig')

## 총 41개 변경

In [130]:
nan_director_modified = pd.read_csv("data_ver5_2.csv")

In [132]:
nan_director_modified

,영화명,개봉일,매출액,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,배급사,등급,장르,감독,배우
0,나의 고향,NaN,143,143,1,1,1,1,폴란드,NaN,NaN,애니메이션,개봉연도 없음,NaN
1,디스턴스,NaN,143,143,1,1,1,1,폴란드,NaN,NaN,애니메이션,개봉연도 없음,NaN
2,문명사회,NaN,143,143,1,1,1,1,불가리아,NaN,NaN,애니메이션,없음,NaN
3,바다의 법,NaN,143,143,1,1,1,1,불가리아,NaN,NaN,애니메이션,없음,NaN
4,비,NaN,143,143,1,1,1,1,불가리아,NaN,NaN,애니메이션,개봉연도 없음,NaN
